# BitArray

Um bequem mit den einzelnen Bits zu operieren, wird eine Klasse `BitArray` erstellt, welche den Wert als `int` abspeichert und Methoden zur Auslesung, Überschreibung, Sequenzierung und Konkatenation ermöglicht.

In [1]:
from math import ceil


class BitArray:
    def __init__(self, bits=None, length=None, file=None):
        if bits is None:
            self.array = 0
            self.length = 0
            if file is not None:
                self.write(file)
        elif type(bits) in {chr, str}:
            self.array = int(bits, 2)
            self.length = len(bits)
            if file is not None:
                self.write(file)
        elif type(bits) == int:
            self.array = bits
            self.length = length if length else bits.bit_length()
            if file is not None:
                self.write(file)
        elif type(bits) == BitArray:
            self.array = bits.array
            self.length = bits.length
            if file is not None:
                self.write(file)
        elif type(bits) == bytes:
            temp = BitArray(int.from_bytes(bits, 'big'))
            cutoff = int(temp[:8])
            self.array = int(temp[8 + cutoff:])
            self.length = len(temp) - (8 + cutoff)
            del temp
            if file is not None:
                self.write(file)
        elif bits is None and file is not None:
            try:
                self.__init__(self.read(file))
            except:
                raise Exception(f'File {file} is invalid!')
        else:
            raise Exception(f"""
Bits must be of type str or int.
{type(bits)} is something completly different.
""")
    
    def __call__(self, step=1):
        for i in range(0, len(self), step):
            if i+step <= len(self):
                yield self[i:i+step]
    
    def __getitem__(self, idx):
        if type(idx) == slice:
            start = (idx.start if idx.start is not None else 0) % len(self)
            stop = (idx.stop if idx.stop is not None else len(self)) % (len(self) + 1)
            return BitArray((self.array >> len(self) - stop) & ((1 << stop - start) - 1),
                            length=stop-start)
        else:
            return BitArray(self.array & 1 << (len(self) - idx - 1) , length=1)
    
    def __setitem__(self, idx, value):
        if type(value) != BitArray:
            raise Exception(f"""
Value must be of type BitArray.
{type(value)} is something completly different.
""")
        cutoff = min(len(self), idx + len(value))
        self.array = (self[:idx]|value[:cutoff]|self[cutoff:]).array
    
    def __or__(self, other):
        return BitArray(self.array << len(other)|other.array, length=len(self) + len(other))
    
    def __repr__(self):
        return f'BitArray[{str(self)}]'
    
    def __str__(self):
        if self.array is not None:
            return bin(self.array).replace('0b', '').zfill(len(self))[:len(self)]
        else:
            return ''
    
    def bytes(self):
        return (BitArray(-len(self) % 8, length=8)|self).array.to_bytes(ceil(len(self)/8) + 1, 'big')
    
    def read(self, file):
        with open(file, 'rb') as file:
            return BitArray(file.read())
    
    def write(self, file):
        with open(file, 'wb') as file:
            file.write(self.bytes())
    
    def __len__(self):
        return self.length
    
    def __int__(self):
        return self.array
    
    def __bool__(self):
        return bool(self.array)
    
    def __eq__(self, other):
        return self.array == other.array and len(self) == len(other)

# Morse Code

Die Morsekodierung bildet von der Menge des Alphabets auf eine Menge bestehend aus weniger Zeichen ab. Für die Kodierung eines einzelnes Zeichens gilt:

$$ \Omega := Alphabet \cup Numerals \cup  \{ {. \_} \}$$
$$ \Sigma :=  \{ .  \_ : *   \} $$
$$ \mathcal{M}: \Omega \mapsto Morse \: Code \subset \Sigma ^{+}$$

Um die Kodierung mehrerer Zeichen $\mathcal{M}^n$ zu ermöglichen, muss eine Trennung der einzelnen Zeichen aus $Morse \: Code$ erfolgen, um die Eindeutligkeit zu gewährleisten. Dies wird auf der Ebene des Morsecodes durch eine Trennung per Leerzeichen `SPACE` zur Abgrenzung der einzelnen Alphabetsbuchstaben ermöglicht:

Sei $\prod$ eine Konkatenation, dann gilt für nicht elementare Strings:
$$ \mathcal{M}(c \in \Omega):= \prod_{i=1}^{|c|}Morse \: Code(c_i) $$

In [2]:
class Encoder:
    def __init__(self):
        self.morse = {
            'A': '._', 'B': '_...', 
            'C': '_._.', 'D': '_..', 
            'E': '.', 'F': '.._.', 
            'G': '__.', 'H': '....', 
            'I': '..', 'J': '.___', 
            'K': '_._', 'L': '._..', 
            'M': '__', 'N': '_.', 
            'O': '___', 'P': '.__.', 
            'Q': '__._', 'R': '._.', 
            'S': '...', 'T': '_', 
            'U': '.._', 'V': '..._', 
            'W': '.__', 'X': '_.._', 
            'Y': '_.__', 'Z': '__..', 
            '0': '_____', '1': '.____', 
            '2': '..___', '3': '...__', 
            '4': '...._', '5': '.....', 
            '6': '_....', '7': '__...', 
            '8': '___..', '9': '____.', 
            ' ': ':', '.': '*',
        }
        self.morse_inv = {v: k for k, v in self.morse.items()}
    
    def encode(self, x, status=False):
        if set(x) - {'.', '_', ':', '*'}:
            x = self.encode_morse(x)
            if status: print(x)
        x = self.encode_binary(x)
        return x

    def decode(self, x, status=False):
        if isinstance(x, BitArray):
            x = self.decode_binary(x)
            if status: print(x)
        x = self.decode_morse(x)
        return x
    
    def encode_morse(self, text):
        return ' '.join(self.morse[char] for char in text.upper() if char in self.morse)

    def decode_morse(self, text):
        return ''.join(self.morse_inv[code] for code in text.split(' ') if code in self.morse_inv)
    
    def morse_ascii(self, text):
        b = BitArray()
        for char in filter(lambda c: c in self.morse, text.upper()):
            b = b|BitArray(ord(char), length=7)
        return b
    
    def compare(self, text):
        ascii_encoding = self.morse_ascii(text)
        stream = self.encode(text)
        print(f"""
Length of Text->ASCII encoding:\t\t{len(ascii_encoding)} bit
Length of Morse->Binary encoding:\t{len(stream)} bit
Redutcion by:\t\t\t\t{round((1 - len(stream)/len(ascii_encoding))*100, 1)}%
""")
    
    def read(self, file, status=False, transform=True):
        with open(file, 'rb') as file:
            x = BitArray(file.read())
            if transform:
                return self.decode(x, status)
            else:
                return x
    
    def write(self, x, file, status=False):
        with open(file, 'wb') as file:
            if isinstance(x, BitArray):
                file.write(x.bytes())
            else:
                file.write(self.encode(x, status).bytes())



### Erster Ansatz


Die dadurch erzeugte Folge kann nun weiter binär kodiert werden. Hierzu werden die Elemente aus $\Sigma$ auf 2 Bit abgebildet.

$$ \varphi : \Sigma \mapsto \{0, 1\}^2$$
$$ \Phi(m) := \prod_{i=1}^{|m|} \varphi(m_i)$$

Da in $\mathcal{M}^n$ jedoch noch das Zeichen `SPACE` enthalen ist, muss eine binäre Kodierung für $\mathcal{M}^n$ die Abbildung $\Phi$ erweitern. Eine naive Methode wäre es, $\Sigma$ um $\_$ zu erweitern, dies hätte jedoch den nachteil, dass wir dann 3 Bit für die Abbildung $\mathcal{B}$ bräuchten. Da $\forall c \in \Omega: |\mathcal{M}(c)| \in [1, 5]$, können wir für jedes `SPACE` $\in \mathcal{M}^n$ die Schritweite $\Lambda$ bis zum nächten `SPACE` mit 3 Bit kodieren.

$$\Lambda: \Sigma \mapsto \{0, 1\}^3$$
$$\Lambda(m) := 2\cdot(|m|-1)_{bin}$$

Damit kann beim Dekodieren $\mathcal{\bar{B}}^m$ zwischen den Schritten des Auslesens der Länge des nächsten Blocks $\Lambda(m)$ und dem Auslesen des nächsten Blocks $\bar{\Phi}$ alterniert werden:

$$\mathcal{B}(x) := \prod_{i = 1}^{|x|}\left[\Lambda|\Phi\right]\circ\mathcal{M}(x_i) $$

In [3]:
class EncoderOne(Encoder):
    def __init__(self):
        super().__init__()
        self.binary = {'.': 0b00, '_': 0b01, ':': 0b10, '*': 0b11,}
        self.binary_inv = {v: k for k, v in self.binary.items()}
    
    def encode_binary(self, codes):
        stream = BitArray()
        for code in codes.split(' '):
            stream = stream|BitArray(len(code)-1, 3)
            for char in code:
                stream = stream|BitArray(self.binary[char], length=2)
        return stream
    
    def decode_binary(self, stream):
        chars, i = [], 0
        while i < len(stream):
            char_length = (int(stream[i:i+3])+1)*2
            chars.append(''.join(self.binary_inv[int(x)] for x in stream[i+3:i+3+char_length](2)))
            i += 3 + char_length 
        return ' '.join(chars)

In [4]:
witch = """
Tell me, what do you do with witches. 
Burn Burn Burn... 
What do you burn apart from witches. 
More witches... 
Wood. 
So, why do witches burn. 
Cause they're made of... wood. 
So, how do we tell if she is made of wood. 
Build a bridge out of her. 
Ahh, but can you not also make bridges out of stone. 
Oh yeah... 
Does wood sink in water. 
No. It floats. 
Let's throw her into the bog. 
What also floats in water. 
Bread. 
Apples. 
Very small rocks. 
Cider. 
Grape gravy. 
Cherries. 
Mud. 
A Duck. 
Exactly. So, logically... 
If she ways the same as a duck... she's made of wood. 
And therefore... A witch. 
We shall use my largest scales. 
"""

one = EncoderOne()
stream = one.encode(witch, True)
print(repr(stream))

_ . ._.. ._.. : __ . : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _.. ___ : .__ .. _ .... : .__ .. _ _._. .... . ... * : _... .._ ._. _. : _... .._ ._. _. : _... .._ ._. _. * * * : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _... .._ ._. _. : ._ .__. ._ ._. _ : .._. ._. ___ __ : .__ .. _ _._. .... . ... * : __ ___ ._. . : .__ .. _ _._. .... . ... * * * : .__ ___ ___ _.. * : ... ___ : .__ .... _.__ : _.. ___ : .__ .. _ _._. .... . ... : _... .._ ._. _. * : _._. ._ .._ ... . : _ .... . _.__ ._. . : __ ._ _.. . : ___ .._. * * * : .__ ___ ___ _.. * : ... ___ : .... ___ .__ : _.. ___ : .__ . : _ . ._.. ._.. : .. .._. : ... .... . : .. ... : __ ._ _.. . : ___ .._. : .__ ___ ___ _.. * : _... .._ .. ._.. _.. : ._ : _... ._. .. _.. __. . : ___ .._ _ : ___ .._. : .... . ._. * : ._ .... .... : _... .._ _ : _._. ._ _. : _.__ ___ .._ : _. ___ _ : ._ ._.. ... ___ : __ ._ _._ . : _... ._. .. _.. __. . ... : ___ .._ _ : ___ .._. : ... _ ___ _. . * : ___ .... : _.__ . ._ .... * * * : _.. ___ . ... : .__ ___

In [5]:
text = one.decode(stream)
print(text)

TELL ME WHAT DO YOU DO WITH WITCHES. BURN BURN BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [6]:
file_name = 'witch_one.b'
one.write(witch, file_name)
print(one.read(file_name))

IO WITH WITCHES. BURN BURN BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [7]:
one.compare(witch)


Length of Text->ASCII encoding:		4256 bit
Length of Morse->Binary encoding:	4552 bit
Redutcion by:				-7.0%



### Zweiter Ansatz
Ein schlechteres Ergebnis als ASCII bei deutlich geringerem Zeichensatz ist noch nicht zufriedenstellend.
Da die von $\mathcal{M}$ kodierten Strings deutlich häufiger Symbole aus der Menge $\{. \_ \}$ beinhalten als aus der Menge $\{* :\}$ und zwischen zwei `SPACE` lediglich Symbole aus nur einer der beiden Mengen stehen, bietet es sich an, im Präfix eines Blocks zu markieren, aus welcher Menge der folgende Block besteht. Dazu wird ein Bit hinzugefügt und jedes Symbol des folgenden Blocks kann mit einem Bit weniger dargestellt werden. Da jeder Block mindestens ein Symbol lang ist, ist eine gleichbleibende oder verbesserte Reduktion der Kodierung gewiss.

Es sei die Kodierung für die einzelnen Elemente aus $\mathcal{M}$:
$$ \varphi : \Sigma \mapsto \{0, 1\}$$
$$ \varphi(m) := \left\{\begin{matrix}
 0 & \text{falls} & m \in \{. :\} \\ 
 1 & \text{falls} & m \in \{\_ * \} \\ 
\end{matrix}\right. $$
Ferner sei:
$$ \Phi(m) := \prod_{i=1}^{|m|}\varphi(m_i) $$

Es sei die Blocklängenfunktion $\Lambda$:
$$\Lambda: \Sigma^+ \mapsto \{0, 1\}^3$$
$$\Lambda(m) := |m|_{bin} - 1$$

Es sei die Funktion $\Psi$ zur Determinierung, um welchen Block es sich handeln wird:
$$\Psi: \Sigma^+ \mapsto \{0, 1\}$$
$$\Psi(m) :=  \left\{\begin{matrix}
 0 & \text{falls} & m \setminus  \{. \_\} = \{\} \\ 
 1 & \text{falls} & m \setminus  \{: * \}  = \{\}\\ 
\end{matrix}\right. $$

Und die gesamte Kodierung:


$$\mathcal{B}(x) := \prod_{i = 1}^{|x|} \left[\Psi|\Lambda|\Phi\right] \circ\mathcal{M}(x_i)$$

In [8]:
class EncoderTwo(Encoder):
    def encode_binary(self, codes):
        stream = BitArray()
        for x in codes.split(' '):
            stream = stream|self.psi(x)|self.lmbda(x)|self.phi(x)
        return stream
    

    def decode_binary(self, stream):
        chars, i = [], 0
        while i < len(stream):
            letter = not bool(stream[i])
            char_length = int(stream[i+1:i+4])+1
            chars.append(''.join(self.phi_inv(stream[i+4:i+4+char_length], letter)))
            i += 4 + char_length 
        return ' '.join(chars)
    
    def psi(self, text):
        if not set(text) - {'.', '_'}:
            return BitArray('0')
        elif not set(text) - {'*', ':'}:
            return BitArray('1')
    
    def phi(self, text):
        b = BitArray()
        for char in text:
            if char in {'.', ':'}:
                b = b|BitArray('0')
            elif char in {'_', '*'}:
                b = b|BitArray('1')
        return b
    
    def phi_inv(self, stream, letter):
        return ''.join((['.', '_'] if letter else [':', '*'])[int(bit)] for bit in stream())
    
    def lmbda(self, text):
        return BitArray(len(text) - 1, length=3)

In [9]:
two = EncoderTwo()
stream = two.encode(witch, True)
print(repr(stream))

_ . ._.. ._.. : __ . : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _.. ___ : .__ .. _ .... : .__ .. _ _._. .... . ... * : _... .._ ._. _. : _... .._ ._. _. : _... .._ ._. _. * * * : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _... .._ ._. _. : ._ .__. ._ ._. _ : .._. ._. ___ __ : .__ .. _ _._. .... . ... * : __ ___ ._. . : .__ .. _ _._. .... . ... * * * : .__ ___ ___ _.. * : ... ___ : .__ .... _.__ : _.. ___ : .__ .. _ _._. .... . ... : _... .._ ._. _. * : _._. ._ .._ ... . : _ .... . _.__ ._. . : __ ._ _.. . : ___ .._. * * * : .__ ___ ___ _.. * : ... ___ : .... ___ .__ : _.. ___ : .__ . : _ . ._.. ._.. : .. .._. : ... .... . : .. ... : __ ._ _.. . : ___ .._. : .__ ___ ___ _.. * : _... .._ .. ._.. _.. : ._ : _... ._. .. _.. __. . : ___ .._ _ : ___ .._. : .... . ._. * : ._ .... .... : _... .._ _ : _._. ._ _. : _.__ ___ .._ : _. ___ _ : ._ ._.. ... ___ : __ ._ _._ . : _... ._. .. _.. __. . ... : ___ .._ _ : ___ .._. : ... _ ___ _. . * : ___ .... : _.__ . ._ .... * * * : _.. ___ . ... : .__ ___

In [10]:
text = two.decode(stream)
print(text)

TELL ME WHAT DO YOU DO WITH WITCHES. BURN BURN BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [11]:
file_name = 'witch_two.b'
two.write(witch, file_name)
print(two.read(file_name))

VE74VENEE . BURN BURN BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [12]:
two.compare(witch)


Length of Text->ASCII encoding:		4256 bit
Length of Morse->Binary encoding:	3796 bit
Redutcion by:				10.8%



### Dritter Ansatz
Bislang sind immernoch einige redundante Informationen enthalten. Zum Beispiel ist bereits bekannt, dass das Folgende Symbol die Länge 1 hat, wenn es sich um ein Satzzeichen handelt. $$\left[\Psi(m) = 1\right] \rightarrow \left[ \Lambda(m) = 1\right]$$
Wir erstellen eine Tabelle für verschiedene Präfixe, welche uns die Information über Länge $\Lambda$ und Symbolart $\Gamma$ kodieren. 

$$ \Gamma(m) := \left\{ \begin{matrix}
0 & \text{falls} &  m \in Alphabet\cup Numerals \\
1 & \text{falls} & m \in \{ * : \} \\
\end{matrix} \right. $$

Da zwei Bits im Präfix unbelegt sind, bilden wir darin direkt die beiden Optionen für die Satzzeichen mit ab.

|Code|$\Gamma$|$\Lambda$|$m$|
|----|---------|--------|---|
|$000$|0|1|?|
|$001$|0|2|?|
|$010$|0|3|?|
|$011$|0|4|?|
|$100$|0|5|?|
|$101$|1|1|`:`|
|$110$|1|1|`*`|

In [13]:
class EncoderThree(Encoder):
    def __init__(self):
        super().__init__()
        self.prefix = {
            (True, 1, None): 0,
            (True, 2, None): 1,
            (True, 3, None): 2,
            (True, 4, None): 3,
            (True, 5, None): 4,
            (False, 1, ':'): 5,
            (False, 1, '*'): 6,
        }
        self.prefix_inv = {v: k for k, v in self.prefix.items()}

    def encode_binary(self, codes):
        stream = BitArray()
        for x in codes.split(' '):
            stream = stream|BitArray(self.prefix[
                not bool(set(x)-{'_', '.'}),
                len(x),
                x if (punctuation := x in {'*', ':'}) else None,
            ], length=3)
            if not punctuation:
                stream = stream|self.phi(x)
        return stream

    def decode_binary(self, stream):
        chars, i = [], 0
        while i < len(stream):
            letter, char_length, punctuation = self.prefix_inv[int(stream[i:i+3])]
            if letter:
                chars.append(''.join(self.phi_inv(stream[i+3:i+3+char_length])))
                i += 3 + char_length 
            else:
                chars.append(punctuation)
                i += 2 + char_length
        return ' '.join(chars)
    
    def phi(self, text):
        b = BitArray()
        for char in text:
            if char == '.':
                b = b|BitArray('0')
            elif char == '_':
                b = b|BitArray('1')
        return b
    
    def phi_inv(self, stream):
        return ''.join(['.', '_'][int(bit)] for bit in stream())

In [14]:
three = EncoderThree()
stream = three.encode(witch, True)
print(repr(stream))

_ . ._.. ._.. : __ . : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _.. ___ : .__ .. _ .... : .__ .. _ _._. .... . ... * : _... .._ ._. _. : _... .._ ._. _. : _... .._ ._. _. * * * : .__ .... ._ _ : _.. ___ : _.__ ___ .._ : _... .._ ._. _. : ._ .__. ._ ._. _ : .._. ._. ___ __ : .__ .. _ _._. .... . ... * : __ ___ ._. . : .__ .. _ _._. .... . ... * * * : .__ ___ ___ _.. * : ... ___ : .__ .... _.__ : _.. ___ : .__ .. _ _._. .... . ... : _... .._ ._. _. * : _._. ._ .._ ... . : _ .... . _.__ ._. . : __ ._ _.. . : ___ .._. * * * : .__ ___ ___ _.. * : ... ___ : .... ___ .__ : _.. ___ : .__ . : _ . ._.. ._.. : .. .._. : ... .... . : .. ... : __ ._ _.. . : ___ .._. : .__ ___ ___ _.. * : _... .._ .. ._.. _.. : ._ : _... ._. .. _.. __. . : ___ .._ _ : ___ .._. : .... . ._. * : ._ .... .... : _... .._ _ : _._. ._ _. : _.__ ___ .._ : _. ___ _ : ._ ._.. ... ___ : __ ._ _._ . : _... ._. .. _.. __. . ... : ___ .._ _ : ___ .._. : ... _ ___ _. . * : ___ .... : _.__ . ._ .... * * * : _.. ___ . ... : .__ ___

In [15]:
text = three.decode(stream)
print(text)

TELL ME WHAT DO YOU DO WITH WITCHES. BURN BURN BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [16]:
file_name = 'witch_three.b'
three.write(witch, file_name)
print(three.read(file_name))

BURN... WHAT DO YOU BURN APART FROM WITCHES. MORE WITCHES... WOOD. SO WHY DO WITCHES BURN. CAUSE THEYRE MADE OF... WOOD. SO HOW DO WE TELL IF SHE IS MADE OF WOOD. BUILD A BRIDGE OUT OF HER. AHH BUT CAN YOU NOT ALSO MAKE BRIDGES OUT OF STONE. OH YEAH... DOES WOOD SINK IN WATER. NO. IT FLOATS. LETS THROW HER INTO THE BOG. WHAT ALSO FLOATS IN WATER. BREAD. APPLES. VERY SMALL ROCKS. CIDER. GRAPE GRAVY. CHERRIES. MUD. A DUCK. EXACTLY. SO LOGICALLY... IF SHE WAYS THE SAME AS A DUCK... SHES MADE OF WOOD. AND THEREFORE... A WITCH. WE SHALL USE MY LARGEST SCALES. 


In [17]:
three.compare(witch)


Length of Text->ASCII encoding:		4256 bit
Length of Morse->Binary encoding:	3025 bit
Redutcion by:				28.9%

